In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class UNetDown(nn.Module):
    def __init__(self, in_size, out_size, normalize=True, dropout=0.0):
        super(UNetDown, self).__init__()
        layers = [nn.Conv2d(in_size, out_size, 3, stride=2, padding=1, bias=False)]
        if normalize:
            layers.append(nn.BatchNorm2d(out_size, 0.8))
        layers.append(nn.LeakyReLU(0.2))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

class RV_Model(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
#         channels_in, self.h, self.w = in_shape
#         channels_out, _, _ = out_shape

#         self.fc    = nn.Linear(latent_dim, self.h * self.w)

        self.down1 = UNetDown(2, 64, normalize=False)
        self.down2 = UNetDown(64, 128)
        self.down3 = UNetDown(128, 256)
        self.down4 = UNetDown(256, 512)
        self.down5 = UNetDown(512, 512)
        self.down6 = UNetDown(512, 512)
        self.down7 = UNetDown(512, 1, normalize=False)
        
        self.final = nn.Sequential(
            nn.Conv2d(512, 1, 3, stride=1, padding=1), nn.Tanh()
        )
        
        
    def forward(self, x):
        # Propogate noise through fc layer and reshape to img shape
#         z = self.fc(z).view(z.size(0), 1, self.h, self.w)
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        d7 = self.down7(d6)
        
        
        return self.final(d7)

In [5]:
from torch.utils.data import Dataset
import glob

In [6]:
from astropy.io import fits

In [11]:
class RV_Dataset(Dataset):
    def __init__(self, directory):
        # Cutting image to parts should happen as a preprocess
        # Convert to torch tensor
        # Normalize IDK
        # NO RESIZE
        self.transform = transforms.ToTensor(),#transforms.Compose(
#             [
# #                 transforms.CenterCrop(size),
# #                 transforms.Resize((size // ratio, size // ratio), Image.BICUBIC),
#                 transforms.ToTensor(),
# #                 transforms.Normalize(mean, std),
#             ]
#         )
        self.files = sorted(glob.glob(directory))

    def __getitem__(self, index):
        img    = fits.open(self.files[index])
        target_rv = img.header['RV']
        # maybe organize data like this
        img_sci = self.transform(img[0].data)
        img_ref = self.transform(img[1].data)
        out = torch.cat((img_sci,img_ref),0)
        return { 'out': out ,'RV': target_rv}
    
    
    def __len__(self):
        return len(self.files)


In [12]:
from astropy.nddata import Cutout2D
from astropy import units as u

In [13]:
import os.path

In [14]:
def preprocess(file_names,outdir,size):
    files = sorted(glob.glob(file_names))
    for file_name in files:
        tbl = fits.open(file_name)
        img_shape = tbl[0].data.shape
        integer = (img_shape[1]//size) + 1
        delta_x = (img_shape[1] - size)/integer
        for i in range(integer):
            position = (img_shape[0]/2, size/2 + i * delta_x)
            size     = (shape[0], size)
            sci   = Cutout2D(tbl['sci'], position, size)
            ref   = Cutout2D(tbl['ref'], position, size)
            
            sci_hdu = fits.ImageHDU(data=sci)
            ref_hdu = fits.ImageHDU(data=ref)
            
            hdu = fits.HDUList([sci_hdu,ref_hdu])
            name = path.join(outdir,path.split(file_name)[1] + '_c{}.fits'.format(i))
            hdu.header['RV'] = tbl['RV']
            hdu.writeto(name)

In [ ]:
lr = 
b1 = 
b2 = 

In [ ]:
model = RV_Model()
mse_loss = torch.nn.L2Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(b1, b2))

In [ ]:
directory = something
dataloader = DataLoader(
    RV_Dataset(directory),
    batch_size=batch_size,
    shuffle=True,
    num_workers=opt.n_cpu,
)

In [16]:
import time
import datetime

In [17]:
prev_time = time.time()
for epoch in range(n_epochs):
    for i, batch in enumerate(dataloader):
        imgs = batch['out']
        target_rv = batch['RV']
        
        optimizer.zero_grad()
        something = model(imgs)
        y = torch.mean(something)
        loss = mse_loss(y,target_rv)
        loss.backward()
        optimizer.step()
        
        print()
        
        batches_done = epoch * len(dataloader) + i
        batches_left = n_epochs * len(dataloader) - batches_done
        time_left = datetime.timedelta(seconds=batches_left * (time.time() - prev_time))
        prev_time = time.time()

        # Print log
        sys.stdout.write(
            "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f] ETA: %s"
            % (
                epoch,
                n_epochs,
                i,
                len(dataloader),
                loss.item(),
                time_left,
            )
        )

NameError: name 'n_epochs' is not defined